In [ ]:
#Import libraries
import geopandas as gpd
import pandas as pd
from math import cos, sin, radians
from shapely.geometry import Point, LineString
from multiprocessing import Pool
from pqdm.processes import pqdm
import logging
from joblib import Parallel, delayed

In [2]:
#Read in polygon or line data to be used for distance calculation

polygon_layer = gpd.read_file('C:/Users/cowboy/Documents/width-use-test/Sample_Data/Buildings.shp')

In [3]:
# Read in your fishent centroid shapefiles generated from CreateFishnet script

point_layer = gpd.read_file('C:/Users/cowboy/Documents/width-use-test/Sample_Data/FishnetCentroids30m.shp')

In [4]:
#Check your projection and make sure it's in meters
polygon_layer = polygon_layer.to_crs(epsg=32612) #Make sure it's in meters
point_layer = point_layer.to_crs(epsg=32612) #Make sure it's in meters

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
max_distance = 5000 # Adjust this value depending on the maximum distance for the line

def process_point(point_idx, point_gdf, increment, max_distance, sindex, polygon_gdf):
    logging.info(f"Processing point index {point_idx}...")
    
    # Generate angles and offsets for search
    angles = range(0, 360, increment)
    xy_offsets = [(max_distance * cos(radians(angle)), max_distance * sin(radians(angle))) for angle in angles]
    
    # Get the geometry of the point
    point_a = point_gdf.loc[point_idx].geometry
    
    # Check for nearby polygons using buffer
    buffer = point_a.buffer(max_distance)
    possible_matches = polygon_gdf.iloc[list(sindex.intersection(buffer.bounds))]
    precise_matches = possible_matches[possible_matches.intersects(buffer)]
    
    # If no matches, return default values
    if precise_matches.empty:
        return (point_a, [(angle, None, None, max_distance) for angle in angles], [(angle, max_distance, (angle + 180) % 360, max_distance) for angle in angles])
    
    # Process each angle
    point_results, opposite_results = [], []
    for angle, (x_offset, y_offset) in zip(angles, xy_offsets):
        endpoint = Point(point_a.x + x_offset, point_a.y + y_offset)
        line = LineString([point_a, endpoint])
        
        # Calculate distances to intersections
        intersections = precise_matches[precise_matches.intersects(line)].geometry.intersection(line)
        distances = intersections.distance(point_a)
        
        # Determine closest intersection
        if not distances.empty:
            min_distance_idx = distances.idxmin()
            closest_distance = distances[min_distance_idx]
            closest_intersection = intersections[min_distance_idx]
            if closest_intersection.geom_type == 'LineString':
                closest_intersection = closest_intersection.interpolate(closest_intersection.project(point_a))
        else:
            closest_distance, closest_intersection = max_distance, None
        
        point_results.append((angle, line, closest_intersection, closest_distance))
    
    # Calculate opposite distances in the same loop
    for angle_idx, (angle, _, _, distance) in enumerate(point_results):
        opposite_angle = (angle + 180) % 360
        opposite_distance = point_results[(angle_idx + len(angles) // 2) % len(angles)][3]
        opposite_results.append((angle, distance, opposite_angle, opposite_distance))
    
    return (point_a, point_results, opposite_results)

# Main function for processing minimum distances
def calculate_minimum_distances_batched(point_layer, polygon_layer, increment, max_distance, batch_size=500):
    point_gdf = gpd.GeoDataFrame(geometry=point_layer.geometry) if not isinstance(point_layer, gpd.GeoDataFrame) else point_layer
    polygon_gdf = gpd.GeoDataFrame(geometry=polygon_layer.geometry) if not isinstance(polygon_layer, gpd.GeoDataFrame) else polygon_layer
    sindex = polygon_gdf.sindex
    total_points = len(point_gdf)
    
    logging.info(f"Total points to process: {total_points}")
    
    # Process points in parallel
    results = Parallel(n_jobs=8)(
        delayed(process_point)(point_idx, point_gdf, increment, max_distance, sindex, polygon_gdf)
        for point_idx in range(total_points)
    )
    
    # Flatten results and create DataFrame
    data = [
        (point_idx + 1, point, line, intersection_point, angle, distance, opposite_angle, opposite_distance)
        for point_idx, (point, point_results, opposite_results) in enumerate(results)
        for (angle, line, intersection_point, distance), (_, _, opposite_angle, opposite_distance) in zip(point_results, opposite_results)
    ]
    df = pd.DataFrame(data, columns=['ID', 'Point', 'Line', 'Intersection Point', 'Angle', 'Distance to Intersection', 'Opposite Angle', 'Opposite Distance'])
    
    # Calculate total distance and find minimum distances
    df['Total Distance'] = df['Distance to Intersection'] + df['Opposite Distance']
    final_results = df.loc[df.groupby('ID')['Total Distance'].idxmin()].reset_index(drop=True)
    
    logging.info("Processing completed!")
    return final_results



In [ ]:
result_df = calculate_minimum_distances_batched(point_layer, polygon_layer, 1, max_distance)

In [ ]:
result_df

In [ ]:
#Export out the centorid with the total minimum distance and angle pair
df2 = result_df.drop('Line', axis=1)
df2 = df2.drop('Intersection Point', axis=1)

gdf = gpd.GeoDataFrame(df2, geometry='Point')

# Export the GeoDataFrame as a shapefile
output_shapefile = 'C:/Users/cowboy/Documents/WidthMetric/Sample_Data/MinimumDistances30m.shp'
gdf.to_file(output_shapefile)